In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from typing import Tuple, List, Optional
import math
import copy
import tqdm
import matplotlib.pyplot as plt

In [28]:
class ConvBlock(nn.Module):
    def __init__(self,
                 in_channels: int,
                 out_channels: int,
                 kernel_size: int,
                 padding: int,
                 batch_norm: bool = False) -> None:
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, 1, padding)
        self.norm = nn.BatchNorm2d(out_channels) if batch_norm else nn.Identity()
        self.relu = nn.LeakyReLU(0.05, inplace=True)

    def forward(self,
                x: torch.Tensor) -> torch.Tensor:
        return self.relu(self.norm(self.conv(x)))

In [51]:
class Discriminator(nn.Module):
    def __init__(self,
                 in_channels: int,
                 out_channels: int,
                 kernel_size: int,
                 padding: int,
                 num_layer: int) -> None:
        super(Discriminator, self).__init__()
        self.head = ConvBlock(in_channels, out_channels, kernel_size, padding)
        self.body = nn.Sequential(
            *[ConvBlock(out_channels, out_channels, kernel_size, padding)
            for _ in range(num_layer)]
        )
        self.tail = nn.Conv2d(out_channels, 1, kernel_size, padding)

    def forward(self,
                x: torch.Tensor) -> torch.Tensor:
        x = self.head(x)
        x = self.body(x)
        return self.tail(x)

In [ ]:
class Generator(nn.Module):
    def __init__(self,
                 in_channels: int,
                 out_channels: int,
                 kernel_size: int,
                 padding: int,
                 num_layers: int,
                 batch_norm: bool = False) -> None:
        super(Generator, self).__init__()
        self.head = ConvBlock(in_channels, out_channels, kernel_size, padding, batch_norm)
        self.body = nn.ModuleList()
        self.body.append(nn.Sequential(
            *[ConvBlock(out_channels, out_channels, kernel_size, padding, batch_norm)
            for _ in range(num_layers)]
        ))
        self.tail = nn.Sequential(
            nn.Conv2d(out_channels, in_channels, kernel_size, padding=padding),
            nn.Tanh()
        )
    
    def init_next_stage(self):
        self.body.append(copy.deepcopy(self.body[-1]))
    
    def forward(self,
                noise: List[torch.Tensor],
                real_shapes: List[torch.Size],
                noise_amp: List[float]) -> torch.Tensor:
        x = self.head(noise[0])
        x = self.body[0](x)
        for i in range(1, len(self.body)):
            x = F.interpolate(x, size=real_shapes[i], mode='bilinear', align_corners=True)
            x = x + self.body[i](x + noise[i] * noise_amp[i])
        return self.tail(x)

In [20]:
def read_image(path: str) -> torch.Tensor:
    image = plt.imread(path)
    image = torch.from_numpy(image).float() / 255.0
    image = image.permute(2, 0, 1)
    return (image - 0.5) * 2

def write_image(path: str,
                image: torch.Tensor) -> None:
    image = (image + 1) / 2
    plt.imsave(path, image.permute(1, 2, 0).cpu().numpy(), vmin=0, vmax=1)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
path = '../data/angkorwat.jpg'
image = read_image(path).to(device)
max_size, min_size = 250, 25
stages = 8 # stop_scale = stages - 1 = 7

In [33]:
def adjust_scales2image(image: torch.Tensor,
                        stop_scale: int = 7) -> Tuple[torch.Tensor, float]:
    # resize the long side of the image to max_size
    scale = min(max_size / max([image.shape[2], image.shape[3]]), 1)
    real = F.interpolate(image, scale_factor=scale, mode='bilinear', align_corners=True)
    # calculate the scaling factor for resizing the short side to min_size
    scale_factor = math.pow(min_size / (min(real.shape[2], real.shape[3])), 1 / stop_scale)
    return real, scale_factor
real, scale_factor = adjust_scales2image(image.unsqueeze(0))
real.shape, scale_factor

(torch.Size([1, 3, 166, 250]), 0.763040197031911)

In [34]:
def create_reals_pyramid(real, scale_factor: float, stop_scale: int = 7):
    reals = []
    for i in range(stop_scale):
        scale = math.pow(scale_factor,((stop_scale-1)/math.log(stop_scale))*math.log(stop_scale-i)+1)
        curr_real = F.interpolate(real, scale_factor=scale, mode='bilinear', align_corners=True)
        reals.append(curr_real)
    reals.append(real)
    return reals

reals = create_reals_pyramid(real, scale_factor)
for r in reals:
    print(r.shape)

torch.Size([1, 3, 25, 37])
torch.Size([1, 3, 28, 42])
torch.Size([1, 3, 33, 49])
torch.Size([1, 3, 39, 60])
torch.Size([1, 3, 50, 76])
torch.Size([1, 3, 71, 107])
torch.Size([1, 3, 126, 190])
torch.Size([1, 3, 166, 250])


In [36]:
torch.randn(1, 1), torch.randn(1), torch.randn(1).item()

(tensor([[0.6290]]), tensor([0.1069]), 0.14044876396656036)

In [63]:
def calc_gradient_penalty(discriminator: Discriminator,
                          real_data: torch.Tensor,
                          fake_data: torch.Tensor,
                          lamb: float = 0.1,
                          device: str = 'cpu') -> torch.Tensor:
    # alpha = torch.rand(1, 1)
    # alpha = alpha.expand(real_data.size())
    alpha = torch.rand(1).item()
    interpolates = alpha * real_data + (1 - alpha) * fake_data
    interpolates.requires_grad_(True)
    disc_interpolates = discriminator(interpolates)
    gradients = torch.autograd.grad(outputs=disc_interpolates, inputs=interpolates,
                                    grad_outputs=torch.ones_like(disc_interpolates, device=device),
                                    create_graph=True, retain_graph=True, only_inputs=True)[0]
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * lamb
    return gradient_penalty

In [90]:
generator = Generator(3, 64, 3, 1, 3, False).to(device)
discriminator = Discriminator(3, 64, 3, 1, 3).to(device)
fixed_noise = []
noise_amp = []


d_iter = g_iter = 3
max_epochs = 2000
lr = 0.0005
lr_scale = 0.1
beta1 = 0.5
gamma = 0.1
lamb = 0.1
alpha = 10
reals_shapes = [real.shape[2:] for real in reals]
train_depth = 3

In [91]:
for stage in range(stages):
    if stage:
        # discriminator.load_state_dict(torch.load(f'output/discriminator/netD.pth'))
        generator.init_next_stage()
    
    real = reals[stage]
    z_opt = reals[0] if stage == 0 else torch.randn(1, 64, *reals_shapes[stage], device=device)
    fixed_noise.append(z_opt)
    
    optim_d = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, 0.999))

    for block in generator.body[:-train_depth]:
        for param in block.parameters():
            param.requires_grad = False
    
    parameter_list = [{"params": block.parameters(), "lr": lr * (lr_scale ** (len(generator.body[-train_depth:]) - 1 - idx))}
            for idx, block in enumerate(generator.body[-train_depth:])]

    # add parameters of head and tail to training
    if stage - train_depth < 0:
        parameter_list += [{"params": generator.head.parameters(), "lr": lr * (lr_scale ** stage)}]
    parameter_list += [{"params": generator.tail.parameters(), "lr": lr}]
    optim_g = torch.optim.Adam(parameter_list, lr=lr, betas=(beta1, 0.999))

    # define learning rate schedules
    schedulerD = torch.optim.lr_scheduler.MultiStepLR(optimizer=optim_d, milestones=[0.8 * max_epochs], gamma=gamma)
    schedulerG = torch.optim.lr_scheduler.MultiStepLR(optimizer=optim_g, milestones=[0.8 * max_epochs], gamma=gamma)

    # calculate noise_amp
    if stage == 0:
        noise_amp.append(1)
    else:
        noise_amp.append(0)
        with torch.no_grad():
            z_reconstruction = generator(fixed_noise, reals_shapes, noise_amp)
        rec_loss = F.mse_loss(z_reconstruction, real)
        noise_amp[-1] = 0.1 * torch.sqrt(rec_loss)

    for epoch in tqdm.tqdm(range(1, max_epochs + 1)):
        noise = [torch.randn(*z.shape, device=device) for z in fixed_noise]

        # update discriminator: minimize D(G(z)) - D(x)
        for j in range(d_iter):
            output = discriminator(real)
            errD_real = -output.mean()
            
            if j == d_iter - 1:
                fake = generator(noise, reals_shapes, noise_amp)
            else:
                with torch.no_grad():
                    fake = generator(noise, reals_shapes, noise_amp)
            
            output = discriminator(fake.detach())
            errD_fake = output.mean()
            
            gradient_penalty = calc_gradient_penalty(discriminator, real, fake.detach(), lamb, device)
            errD_total = errD_real + errD_fake + gradient_penalty
            optim_d.zero_grad()
            errD_total.backward()
            optim_d.step()
        
        # update generator: minimize -D(G(z))
        output = discriminator(fake)
        errG = -output.mean()
        
        rec = generator(fixed_noise, reals_shapes, noise_amp)
        rec_loss = alpha * F.mse_loss(rec, real)
        
        errG_total = errG + rec_loss
        optim_g.zero_grad()
        errG_total.backward()
        for _ in range(g_iter):
            optim_g.step()
        
        if epoch % 250 == 0:
            print(f'Epoch: {epoch: 04d}, D_real Loss: {-errD_real.item():.5f}, D_fake Loss: {errD_fake.item():.5f}, '
                  f'D_gradient_penalty: {gradient_penalty.item():.5f}, G_fake Loss: {errG.item():.5f}, G_rec Loss: {rec_loss.item():.5f}')
        if epoch % 500 == 0:
            write_image(f'output/fake_sample_{epoch:04d}.jpg', fake.detach().squeeze(0))
            write_image(f'output/reconstruction_{epoch: 04d}.jpg', rec.detach().squeeze(0))
        
        schedulerD.step()
        schedulerG.step()

 13%|█▎        | 255/2000 [00:09<01:02, 28.04it/s]

Epoch:  250, D_real Loss: 4.30068, D_fake Loss: 1.59975, D_gradient_penalty: 0.53338, G_fake Loss: -2.47047, G_rec Loss: 0.10092


 25%|██▌       | 503/2000 [00:18<00:53, 28.05it/s]

Epoch:  500, D_real Loss: 3.07716, D_fake Loss: 0.60868, D_gradient_penalty: 0.51457, G_fake Loss: -1.20070, G_rec Loss: 0.07324


 38%|███▊      | 756/2000 [00:27<00:43, 28.73it/s]

...

Epoch:  1500, D_real Loss: -1.36419, D_fake Loss: -1.50761, D_gradient_penalty: 0.04406, G_fake Loss: 1.57757, G_rec Loss: 0.01631


 88%|████████▊ | 1750/2000 [17:25<02:06,  1.97it/s]

Epoch:  1750, D_real Loss: -1.39273, D_fake Loss: -1.52153, D_gradient_penalty: 0.05710, G_fake Loss: 1.53567, G_rec Loss: 0.00886


100%|██████████| 2000/2000 [19:19<00:00,  1.73it/s]

Epoch:  2000, D_real Loss: -1.41666, D_fake Loss: -1.55512, D_gradient_penalty: 0.06552, G_fake Loss: 1.57962, G_rec Loss: 0.00875


In [86]:
F.interpolate(torch.randn(1, 3, 25, 37), size=(28, 42), mode='bilinear', align_corners=True).shape

torch.Size([1, 3, 28, 42])

In [78]:
len(noise)

3

In [79]:
noise[0].shape, noise[1].shape, noise[2].shape

(torch.Size([1, 3, 25, 37]),
 torch.Size([1, 3, 28, 42]),
 torch.Size([1, 3, 25, 37]))

In [75]:
len(fixed_noise)

2

In [76]:
fixed_noise[0].shape, fixed_noise[1].shape

(torch.Size([1, 3, 25, 37]), torch.Size([1, 3, 28, 42]))

In [71]:
noise_amp

[1]